<a href="https://colab.research.google.com/github/sudarshan026/sudarshan/blob/main/Alumni_feedback_catalyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
import pandas as pd
import os
import matplotlib.pyplot as plt
from fpdf import FPDF
import textwrap
import shutil

# === Upload file ===
print("Please upload the Excel file:")
uploaded = files.upload()
if not uploaded:
    print("No file uploaded, exiting...")
    exit()

file_name = list(uploaded.keys())[0]
print(f"Uploaded file: {file_name}")

# === Mode Selection ===
print("\nChoose report mode:\n1. Branch-wise Reports\n2. General Report")
mode = input("Enter 1 or 2: ").strip()

# === Load Data ===
df = pd.read_excel(file_name)
curriculum_cols = [col for col in df.columns if "Curriculum" in col]
facilities_cols = [col for col in df.columns if "Facilities" in col]

# === Prepare output directory ===
output_dir = "reports"
os.makedirs(output_dir, exist_ok=True)

# === Helper Functions ===
def generate_summary_table(sub_df, category_cols):
    summary = {}
    for col in category_cols:
        scores = sub_df[col].dropna().astype(int).value_counts().to_dict()
        summary[col] = {i: scores.get(i, 0) for i in range(1, 5)}
    score_df = pd.DataFrame(summary).T.fillna(0).astype(int)
    score_df["Total"] = score_df[[4, 3, 2, 1]].sum(axis=1)
    for i in range(4, 0, -1):
        score_df[f"% of {i}"] = round(score_df[i] * 100 / score_df["Total"], 2)
    score_df = score_df.reset_index().rename(columns={"index": "Category"})
    return score_df[["Category", "Total", 4, "% of 4", 3, "% of 3", 2, "% of 2", 1, "% of 1"]]

def plot_ratings(score_df, name, title_prefix):
    df_plot = score_df.set_index("Category")[[4, 3, 2, 1]]
    ax = df_plot.plot(kind='bar', figsize=(14, 6))
    for bars in ax.containers:
        ax.bar_label(bars, label_type='edge', fontsize=9)
    plt.title(f"{name} Ratings - {title_prefix}")
    plt.xlabel(name)
    plt.ylabel("No. of Responses")
    wrapped_labels = ['\n'.join(textwrap.wrap(label, width=25)) for label in df_plot.index]
    ax.set_xticklabels(wrapped_labels, rotation=45, ha='center')
    plt.legend(title="Rating")
    plt.tight_layout()
    chart_path = os.path.join(output_dir, f"{title_prefix}_{name}.png".replace(" ", "_"))
    plt.savefig(chart_path, bbox_inches='tight')
    plt.close()
    return chart_path

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Ratings Report', ln=1, align='C')

    def table(self, df, y_start):
        self.set_xy(10, y_start)
        self.set_font('Arial', '', 9)
        first_col_width = 60
        other_col_width = (self.w - 20 - first_col_width) / (len(df.columns) - 1)
        row_height = 6

        self.set_font('Arial', 'B', 9)
        self.cell(first_col_width, row_height, str(df.columns[0]), border=1)
        for col in df.columns[1:]:
            self.cell(other_col_width, row_height, str(col), border=1)
        self.ln()

        self.set_font('Arial', '', 8)
        for i in range(len(df)):
            y_before = self.get_y()
            x_before = self.get_x()
            self.multi_cell(first_col_width, row_height, str(df.iloc[i, 0]), border=1)
            y_after = self.get_y()
            height_used = y_after - y_before
            self.set_xy(x_before + first_col_width, y_before)
            for item in df.iloc[i, 1:]:
                self.cell(other_col_width, height_used, str(item), border=1)
            self.ln()

    def insert_image_with_page_check(self, image_path, y_margin=10):
        current_y = self.get_y()
        estimated_image_height = 80
        if current_y + estimated_image_height > self.h - 20:
            self.add_page()
            current_y = 20
        self.image(image_path, x=10, y=current_y + y_margin, w=self.w - 20)
        self.set_y(current_y + estimated_image_height + y_margin)

# === Report Generation ===

if mode == '1':
    print("Generating branch-wise reports...")
    branches = df['Branch'].dropna().unique()
    for branch in branches:
        branch_df = df[df['Branch'] == branch]
        curriculum_score = generate_summary_table(branch_df, curriculum_cols)
        facilities_score = generate_summary_table(branch_df, facilities_cols)

        chart_curriculum = plot_ratings(curriculum_score, "Curriculum", branch)
        chart_facilities = plot_ratings(facilities_score, "Facilities", branch)

        pdf = PDF()
        pdf.add_page()
        pdf.set_font('Arial', 'B', 11)
        pdf.cell(0, 10, f"Branch: {branch}", ln=1)

        pdf.table(curriculum_score, pdf.get_y() + 5)
        pdf.table(facilities_score, pdf.get_y() + 10)

        pdf.insert_image_with_page_check(chart_curriculum)
        pdf.insert_image_with_page_check(chart_facilities)

        pdf.output(os.path.join(output_dir, f"{branch}_ratings.pdf"))

    shutil.make_archive("branch_reports", 'zip', output_dir)
    files.download("branch_reports.zip")

elif mode == '2':
    print("Generating general report...")
    curriculum_score = generate_summary_table(df, curriculum_cols)
    facilities_score = generate_summary_table(df, facilities_cols)

    chart_curriculum = plot_ratings(curriculum_score, "Curriculum", "Overall")
    chart_facilities = plot_ratings(facilities_score, "Facilities", "Overall")

    pdf = PDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 11)
    pdf.cell(0, 10, f"Overall Feedback Report", ln=1)

    pdf.table(curriculum_score, pdf.get_y() + 5)
    pdf.table(facilities_score, pdf.get_y() + 10)

    pdf.insert_image_with_page_check(chart_curriculum)
    pdf.insert_image_with_page_check(chart_facilities)

    output_pdf_path = os.path.join(output_dir, "overall_ratings.pdf")
    pdf.output(output_pdf_path)
    files.download(output_pdf_path)

else:
    print("Invalid option selected. Please run the code again and choose 1 or 2.")


Please upload the Excel file:


Saving Alumni Feedback Form sample data.xlsx to Alumni Feedback Form sample data (2).xlsx
Uploaded file: Alumni Feedback Form sample data (2).xlsx

Choose report mode:
1. Branch-wise Reports
2. General Report
Enter 1 or 2: 2
Generating general report...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>